In [1]:
import sys
sys.path.append('../src')
from sklearn.impute import SimpleImputer

def qcutting(df,col_names):
    data = df.copy()
    for col in col_names:
        data[col] = pd.qcut(df[col].rank(method='first'), 
                            4, 
                            labels=["0","1","2","3"])
    return data         

In [2]:
from feature_selection import FeatureSelection
from sklearn.preprocessing import OneHotEncoder
from config import Configure
import pandas as pd


print('\n Aplicando algoritmo de seleção de parâmetros')
settings = Configure()
settings.set_fs_params()
settings.set_pre_processing_params()
pp_params = settings.pre_processing_params
fs_params = settings.feature_selection_params
df1 = pd.read_csv(settings.pf1_folder)
df2 = pd.read_csv(settings.pf2_folder)
df3 = pd.read_csv(settings.pf3_folder)
mkt = pd.read_csv(settings.mkt_folder)
fs = FeatureSelection(mkt,  df1, df2, df3, pp_params, fs_params)
values, features = fs.feature_selection_algorithm()
columns = features.values[features.values != 'Unnamed: 0']
id_column = mkt['id']
mkt = mkt[columns]
print("Result approax:", values.max())


 Aplicando algoritmo de seleção de parâmetros
Result approax: 0.8317694307173836


/home/guilherme/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/guilherme/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


/home/guilherme/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:441: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. The passed value of 'all' is the default and can simply be removed.
  DeprecationWarning)
/home/guilherme/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:441: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. The passed value of 'all' is the default and can simply be removed.
  DeprecationWarning)


In [11]:
X

array([[0., 0., 0., ..., 1., 0., 1.],
       [0., 1., 0., ..., 1., 0., 1.],
       [1., 0., 0., ..., 1., 0., 1.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 1.]])

In [ ]:
def df_transform(df):
    columns = df.columns
    for col in columns:
        var_type = df[col].dtype 
        if ((var_type == 'bool')|(var_type == 'object')):
            df[col] = df[col].fillna(df[col].mode().values[0])
        else:
            df[col] = df[col].fillna(df[col].median())
    df = qcutting(df, 
                  columns[(df.dtypes=='float64')|(df.dtypes=='int64')])
    encoder = OneHotEncoder(categorical_features='all',sparse = False)
    X = encoder.fit_transform(df)